In [58]:
import pandas as pd
import numpy as np

In [59]:
# Frame containing all the useful values

fundings = pd.read_excel ('grant.xlsx') [['University', 'Approved Amount']]
fundings.head ()

,University,Approved Amount
0,Nicht zuteilbar - NA,11619.00
1,Université de Genève - GE,41022.00
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00
3,Universität Basel - BS,52627.00
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00


In [9]:
# Cantons keys

cantons = pd.read_csv ('cantons.csv')
cantons.head ()

,Canton
0,ZH
1,BE
2,LU
3,UR
4,SZ


In [66]:
# List of all the universities

universities = pd.DataFrame(fundings ['University'].unique ())

In [ ]:

universities = universities[1:]
universities['Canton'] = universities['University'].apply(get_canton)
universities

In [95]:
def get_canton(university):
    from credentials import places_api_key
    # Get placeid from university name
    try:
        url_text_search = 'https://maps.googleapis.com/maps/api/place/textsearch/json?key={key}&query={query}'.format(
                key=places_api_key,
                query=university
        )
        r=requests.get(url_text_search)
        placeid = r.json()['results'][0]['place_id']

        # Get info on placeid
        url_place_search = 'https://maps.googleapis.com/maps/api/place/details/json?key={key}&placeid={placeid}'.format(
            key=places_api_key,
            placeid=placeid
        )
        r2=requests.get(url_place_search)

        # Parse json and return canton
        for el in r2.json()['result']['address_components']:
            if el['types']==['administrative_area_level_1', 'political']:
                return el['short_name']
    except Exception:
        return None

In [96]:
get_canton('NPO (Biblioth., Museen, Verwalt.) - NPO')

# Tasks left for the homework : 
### - classify each university to its canton, then groupyby canton,
### - sum the approved amount into a column named 'Granted Amount'
### - call the function 'buildChoroplethMap' from the script 'choropleth' to produce the map. The final dataFrame should have two columns 'Canton' and 'Granted Money'.

# Example of use for function 'buildChoroplethMap'

In [13]:
testFrame = pd.merge (cantons, pd.DataFrame (np.random.randint(0,100,size=(100, 1)), columns = ['Granted Money']), left_index = True, right_index = True)
testFrame.head ()

,Canton,Granted Money
0,ZH,22
1,BE,88
2,LU,97
3,UR,52
4,SZ,53


## Parameters :

 - dataFrame : the data frame containing canton values
 - scale : color scales on the map
 - outputFileName : the name of the output map file